# Euler allocation of an exemplary rates portfolio

In [1]:
import QuantLib as ql
from scipy import optimize
from allocation.Enums import FdApproach2
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement, InitialMargining
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.irs import IRS
from instruments.interestRateInstrument.ois import OIS
from instruments.interestRateInstrument.swaption import Swaption
from jupyterUtils import export
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection
import pandas as pd
asdf = 1

We create a portfolio consisting of a big USD payer swap and a smaller EUR receiver swap of same maturity.

In [2]:
payer_usd_6Y = IRS(notional = 1000000000,
                   swapDirection=SwapDirection.PAYER,
                   timeToSwapStart=ql.Period(2, ql.Days),
                   timeToSwapEnd=ql.Period(6, ql.Years),
                   index = InterestRateIndex.USDLIBOR3M)

receiver_eur_6Y = IRS(notional=180000000,
                      swapDirection=SwapDirection.RECEIVER,
                      timeToSwapStart=ql.Period(2,ql.Days),
                      timeToSwapEnd=ql.Period(6,ql.Years),
                      index=InterestRateIndex.EURIBOR6M)

We create three collateral agreements for which only VM but not IM is exchanged. The portfolios associated with the first two collateral agreements only consist of one of the two trades, while the portfolio associated with the third collateral agreement contains both trades.

In [3]:
ca_together_vm_only = CollateralAgreement(initialMargining = InitialMargining.NO_IM)
ca_together_vm_only.link_sa_ccr_instance(SA_CCR(ca_together_vm_only))
ca_together_vm_only.add_trades([payer_usd_6Y, receiver_eur_6Y])

ca_usd_vm_only = CollateralAgreement(initialMargining = InitialMargining.NO_IM)
ca_usd_vm_only.link_sa_ccr_instance(SA_CCR(ca_usd_vm_only))
ca_usd_vm_only.add_trades(payer_usd_6Y)

ca_eur_vm_only = CollateralAgreement(initialMargining = InitialMargining.NO_IM)
ca_eur_vm_only.link_sa_ccr_instance(SA_CCR(ca_eur_vm_only))
ca_eur_vm_only.add_trades(receiver_eur_6Y)

We allocate the EAD of the portfolio containing both trades to the two individual trades.

In [4]:
eulerAllocator = EulerAllocator(ca_together_vm_only)
eulerAllocator.fdApproach2 = FdApproach2.Central
alloc = eulerAllocator.allocate_ead()

We repeat the same process for collateral agreements for which VM and IM is exchanged. In this case, we also allocate the IM in addition to the EAD.

In [6]:
ca_together_vm_im = CollateralAgreement(initialMargining = InitialMargining.SIMM)
ca_together_vm_im.link_sa_ccr_instance(SA_CCR(ca_together_vm_im))
ca_together_vm_im.add_trades([payer_usd_6Y, receiver_eur_6Y])

ca_usd_vm_im = CollateralAgreement(initialMargining = InitialMargining.SIMM)
ca_usd_vm_im.link_sa_ccr_instance(SA_CCR(ca_usd_vm_im))
ca_usd_vm_im.add_trades(payer_usd_6Y)

ca_eur_vm_im = CollateralAgreement(initialMargining = InitialMargining.SIMM)
ca_eur_vm_im.link_sa_ccr_instance(SA_CCR(ca_eur_vm_im))
ca_eur_vm_im.add_trades(receiver_eur_6Y)

eulerAllocator2 = EulerAllocator(ca_together_vm_im)
eulerAllocator2.fdApproach2 = FdApproach2.Central
alloc2 = eulerAllocator2.allocate_ead()

alloc_im = eulerAllocator2.allocate_im()

In [6]:
def trrep(t):
    import ast
    dic = ast.literal_eval(str(t))
    if dic['Instrument'] in ['IRS','OIS']:
        return '{:,.0f}Mn '.format(dic['Notional']/1000000)+t.index.name+' '+t.swapDirection.name+' '+dic['Instrument']+' '+str(t.ql_timeToSwapEnd)
    else:
        return '{:,d}'.format(t.underlying_swap.notional)+' '+t.underlying_swap.index.name+' '+t.underlying_swap.swapDirection.name+' '+dic['Instrument']+' '+str(t.underlying_swap.ql_timeToSwapStart) + ' to ' + str(t.underlying_swap.ql_timeToSwapEnd)

This yields the following results for the calculated EADs and IMs:

In [7]:
pd.options.display.float_format = '{:,.0f} USD'.format
data = {'Title':[trrep(receiver_eur_6Y),
                 trrep(payer_usd_6Y),
                 'Portfolio'],
        '(Standalone) EAD VM only':[ca_eur_vm_only.get_sa_ccr_model().get_ead(),
                                  ca_usd_vm_only.get_sa_ccr_model().get_ead(),
                                  ca_together_vm_only.get_sa_ccr_model().get_ead()],
        '(Standalone) EAD VM + IM':[ca_eur_vm_im.get_sa_ccr_model().get_ead(),
                                  ca_usd_vm_im.get_sa_ccr_model().get_ead(),
                                  ca_together_vm_im.get_sa_ccr_model().get_ead()],
        '(Standalone) IM':[ca_eur_vm_im.get_im_model().get_risk_measure(),
                         ca_usd_vm_im.get_im_model().get_risk_measure(),
                         ca_together_vm_im.get_im_model().get_risk_measure()]}
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df[['(Standalone) EAD VM only','(Standalone) IM','(Standalone) EAD VM + IM']]

,(Standalone) EAD VM only,(Standalone) IM,(Standalone) EAD VM + IM
180Mn EURIBOR6M RECEIVER IRS 6Y,"1,957,315 USD","6,079,460 USD","286,420 USD"
"1,000Mn USDLIBOR3M PAYER IRS 6Y","10,873,970 USD","28,762,683 USD","2,014,873 USD"
Portfolio,"12,831,284 USD","28,059,093 USD","3,074,959 USD"


And the following allocations of the risk measures:

In [8]:
pd.options.display.float_format = '{:.2%}'.format
data = {'Title':[trrep(receiver_eur_6Y),
                 trrep(payer_usd_6Y),
                 'Portfolio'],
        'Allocated EAD VM only':[alloc[receiver_eur_6Y]/sum(alloc.values()),
                                 alloc[payer_usd_6Y]/sum(alloc.values()),
                                 sum(alloc.values())],
        'Allocated IM':[alloc_im[receiver_eur_6Y]/sum(alloc_im.values()),
                        alloc_im[payer_usd_6Y]/sum(alloc_im.values()),
                        sum(alloc_im.values())],
        'Allocated EAD VM + IM':[alloc2[receiver_eur_6Y]/sum(alloc2.values()),
                                 alloc2[payer_usd_6Y]/sum(alloc2.values()),
                                 sum(alloc2.values())]}
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df = df.loc[~df.index.isin(['Portfolio'])]
df

,Allocated EAD VM only,Allocated IM,Allocated EAD VM + IM
180Mn EURIBOR6M RECEIVER IRS 6Y,15.25%,-0.19%,34.95%
"1,000Mn USDLIBOR3M PAYER IRS 6Y",84.75%,100.19%,65.05%


As can be seen the EUR Receiver Swap is considered to increase risk for the EAD risk measure while the IM risk measure considers it to be a hedge position. Further discussion of this result in section \todo{Reference required}. 

In [9]:
# pd.options.display.float_format = '{:,.2f} USD'.format
# df = pd.DataFrame(df.loc['Portfolio']).transpose()
# df.rename(columns = {'Allocated EAD VM only': 'EAD VM only',
#                      'Allocated IM': 'IM',
#                      'Allocated EAD VM + IM':'EAD VM + IM'})

Observations

- EAD under consideration of VM and IM is not subadditive
- EUR IRS is lowering IM but increasing EAD without IM, therefore significantly increasing EAD with VM and IM
- Explaining very different allocations to the EUR IRS 15% of EAD without IM, 40% of EAD with IM, -0.5% of the IM
- Since no hedge effect under SA-CCR between the two trades sum of standalone portfolios = Portfolio allocation and allocated value = standalone EAD

To make it a bit simpler to compare

I want to try to find the optimal hedge notional of the EUR trade to minimize ISDA SIMM

In [10]:
def find_notional_for_minimal_simm(notional):
    eur_trade = IRS(notional=notional,
                      swapDirection=SwapDirection.RECEIVER,
                      timeToSwapStart=ql.Period(2,ql.Days),
                      timeToSwapEnd=ql.Period(6,ql.Years),
                      index=InterestRateIndex.EURIBOR6M)
    ca_together_vm_im.replace_trade(receiver_eur_6Y, eur_trade)
    result = ca_together_vm_im.get_im_model().get_risk_measure()
    ca_together_vm_im.replace_trade(eur_trade, receiver_eur_6Y)
    return result

opt_notional = optimize.minimize_scalar(find_notional_for_minimal_simm,bounds=(5, 100000000), method='bounded')
opt_notional

     fun: 28212680.863763526
 message: 'Solution found.'
    nfev: 37
  status: 0
 success: True
       x: 99999998.48791124

To further showcase the differences between the two risk measures and their potentially unexpected interaction we add another trade to the portfolio.

For this we add a one year Swaption on a five year USD receiver swap to the portfolio and reallocate:

In [11]:
# receiver_usd_30y = OIS(notional=3600000,
#                        swapDirection=SwapDirection.RECEIVER,
#                        timeToSwapStart=ql.Period(2, ql.Days),
#                        timeToSwapEnd=ql.Period(30, ql.Years),
#                        index=InterestRateIndex.FEDFUNDS)
# ca_together_vm_im.add_trades(receiver_usd_30y)
# ca_together_vm_only.add_trades(receiver_usd_30y)
#
# alloc = eulerAllocator.allocate_ead()
# alloc2 = eulerAllocator2.allocate_ead()
# alloc_im = eulerAllocator2.allocate_im()

In [12]:
#
# data = {'Title':[trrep(receiver_eur_6Y),
#                  trrep(payer_usd_6Y),
#                  trrep(receiver_usd_30y),
#                  'Portfolio'],
#         'Allocated EAD VM only':[alloc[receiver_eur_6Y]/sum(alloc.values()),
#                                  alloc[payer_usd_6Y]/sum(alloc.values()),
#                                  alloc[receiver_usd_30y]/sum(alloc.values()),
#                                  sum(alloc.values())],
#         'Allocated IM':[alloc_im[receiver_eur_6Y]/sum(alloc_im.values()),
#                         alloc_im[payer_usd_6Y]/sum(alloc_im.values()),
#                         alloc_im[receiver_usd_30y]/sum(alloc_im.values()),
#                         sum(alloc_im.values())],
#         'Allocated EAD VM + IM':[alloc2[receiver_eur_6Y]/sum(alloc2.values()),
#                                  alloc2[payer_usd_6Y]/sum(alloc2.values()),
#                                  alloc2[receiver_usd_30y]/sum(alloc2.values()),
#                                  sum(alloc2.values())]}
# df = pd.DataFrame.from_dict(data)
# df = df.set_index('Title')
# df.index.name = None
# df

In [13]:
ul_swap = IRS(notional=500000000,
              swapDirection=SwapDirection.RECEIVER,
              timeToSwapStart=ql.Period(1, ql.Years),
              timeToSwapEnd=ql.Period(6, ql.Years),
              index = InterestRateIndex.USDLIBOR3M)
rec_swaption_1_6 = Swaption(underlyingSwap=ul_swap,
                            optionMaturity=ql.Period(1, ql.Years))

ca_together_vm_im.add_trades(rec_swaption_1_6)
ca_together_vm_only.add_trades(rec_swaption_1_6)

alloc = eulerAllocator.allocate_ead()
alloc2 = eulerAllocator2.allocate_ead()
alloc_im = eulerAllocator2.allocate_im()

This yields the following results:

In [14]:
pd.options.display.float_format = '{:.2%}'.format
data = {'Title':[trrep(receiver_eur_6Y),
                 trrep(payer_usd_6Y),
                 # trrep(receiver_usd_30y),
                 trrep(rec_swaption_1_6),
                 'Portfolio'],
        'Allocated EAD VM only':
            [alloc[receiver_eur_6Y]/sum(alloc.values()),
             alloc[payer_usd_6Y]/sum(alloc.values()),
             alloc[rec_swaption_1_6]/sum(alloc.values()),
             sum(alloc.values())],
        'Allocated IM':
            [alloc_im[receiver_eur_6Y]/sum(alloc_im.values()),
             alloc_im[payer_usd_6Y]/sum(alloc_im.values()),
             alloc_im[rec_swaption_1_6]/sum(alloc_im.values()),
             sum(alloc_im.values())],
        'Allocated EAD VM + IM':
            [alloc2[receiver_eur_6Y]/sum(alloc2.values()),
             alloc2[payer_usd_6Y]/sum(alloc2.values()),
             alloc2[rec_swaption_1_6]/sum(alloc2.values()),
             sum(alloc2.values())]}
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df.loc[~df.index.isin(['Portfolio'])]

,Allocated EAD VM only,Allocated IM,Allocated EAD VM + IM
180Mn EURIBOR6M RECEIVER IRS 6Y,17.70%,0.50%,43.69%
"1,000Mn USDLIBOR3M PAYER IRS 6Y",98.33%,80.07%,125.93%
"500,000,000 USDLIBOR3M RECEIVER Swaption 1Y to 6Y",-16.04%,19.44%,-69.62%


In [15]:
pd.options.display.float_format = '{:,.0f} USD'.format
df = pd.DataFrame(df.loc['Portfolio']).transpose()
df.rename(columns = {'Allocated EAD VM only': 'EAD VM only',
                     'Allocated IM': 'IM',
                     'Allocated EAD VM + IM':'EAD VM + IM'})

,EAD VM only,IM,EAD VM + IM
Portfolio,"11,058,114 USD","34,796,088 USD","1,586,748 USD"


We observe that the Swaption is considered a hedge for the SA-CCR EAD risk measure while it increases risk under the ISDA-SIMM IM risk measure.
The displayed results are discussed further in \todo{Reference missing}

Observations:

- Swaption is considered a hedge for EAD while it is increasing IM therefore very efficient in decreasing EAD. Opposite phenomenon

In [16]:
export('SA-CCR Euler allocation of an exemplary rates portfolio.ipynb')